In [4]:
import os
import xarray as xr
import numpy as np

# inputs and outputs
ipath = '/glade/work/mizukami/test_mizuRoute/NHDPlus_testCase/output'
ifile_org = 'runoff_1951-1-1.map.gnu.bmrk.nc'
ifile_pio = 'runoff_1951-1-1.pio.map.gnu.bmrk.nc'

opath = '/glade/work/nijssen/scratch'
ofile = os.path.join(opath, ifile_pio.replace('.nc', '.reorder.nc'))

# open the two datafiles
ds_org = xr.open_dataset(os.path.join(ipath, ifile_org))
ds_pio = xr.open_dataset(os.path.join(ipath, ifile_pio))

# extract the reachID orders
x = ds_pio.reachID.values
y = ds_org.reachID.values

# Find the indices of the reordered array
# From: https://stackoverflow.com/questions/8251541/numpy-for-every-element-in-one-array-find-the-index-in-another-array
index = np.argsort(x)
sorted_x = x[index]
sorted_index = np.searchsorted(sorted_x, y)
remap_order = np.take(index, sorted_index, mode="clip")

# Reorder pio according to the orginal
ds_reorder = ds_pio.isel(dict(seg=remap_order))

# Write to file. reachID should have same order as in the original file
ds_reorder.to_netcdf(ofile)

In [3]:
ds_org.to_netcdf(os.path.join(opath, ifile_org))